In [73]:
from simple_salesforce import Salesforce
import arcgis
import os

In [74]:
# connection to Salesforce
SF = Salesforce("iet@detroitmi.gov", os.environ['SF_PASS'], os.environ['SF_TOKEN'])

In [75]:
# Helper functions
gc = arcgis.geocoding.Geocoder("https://gis.detroitmi.gov/arcgis/rest/services/DoIT/CompositeGeocoder/GeocodeServer")
def geocode_record(record, column):
    res = arcgis.geocoding.geocode(address=record[column], geocoder=gc, out_sr=4326)
    best = res[0]['location']
    return '({}, {})'.format(round(best['y'], 5), round(best['x'], 5))
    
def process_record(record, fieldmap):
    """Create a new record from Salesforce via a field mapping."""
    data = {}
    for k, v in fieldmap.items():
        try:
            data[k] = record[v]
        except: 
            print("no match on field: {}".format(v))
    # attach address data as location
    data['location'] = geocode_record(data, 'Address')
    return data

In [76]:
next_thirty = SF.query_all("""
                           Select Id, 
                            Parcel_ID__c, 
                            Address__c, 
                            DEMO_Planned_Knock_Down_Date__c
                           from Case 
                            WHERE DEMO_Planned_Knock_Down_Date__c = NEXT_N_DAYS:30
                            ORDER BY DEMO_Planned_Knock_Down_Date__c ASC
                            """)

mapping = {
    'Parcel ID': 'Parcel_ID__c',
    'Address': 'Address__c',
    'Projected Demolition Date': 'DEMO_Planned_Knock_Down_Date__c',
}

In [77]:
# get an array of processed SF rows
rows = [process_record(r, mapping) for r in next_thirty['records']]

In [79]:
from sodapy import Socrata
# connection to Socrata
socrata_client = Socrata("data.detroitmi.gov", os.environ['SODA_TOKEN'], os.environ['SODA_USER'], os.environ['SODA_PASS'])

In [80]:
# columns for Next 30 Days demolition dataset
columns = [
    {"fieldName": "parcelnumb", "name": "Parcel ID", "dataTypeName": "text"},
    {"fieldName": "demo_date", "name": "Projected Demolition Date", "dataTypeName": "date"},
    {"fieldName": "address", "name": "Address", "dataTypeName": "text"},
    {"fieldName": "location", "name": "Location", "dataTypeName": "location"}
]

# tags for Next 30 Days demolition dataset
tags = ["DLBA", "demolitions"]

In [81]:
# make the dataset
dataset = socrata_client.create("Demolitions Next 30 Days w Location", 
                      description="Demolitions happening in the next 30 days",
                      columns=columns,
                      row_identifier="parcelnumb",
                      tags=tags,
                      category="Government")

In [82]:
# upsert rows from Salesforce
socrata_client.upsert(dataset['id'], rows)

{'By RowIdentifier': 113,
 'By SID': 0,
 'Errors': 0,
 'Rows Created': 113,
 'Rows Deleted': 0,
 'Rows Updated': 0}